In [1]:
from langchain.agents import initialize_agent, AgentType, Tool
from langchain.memory import ConversationBufferMemory
from langgraph.graph import StateGraph
from langchain.chat_models import ChatOpenAI
from langchain_gigachat.chat_models import GigaChat
from langchain.schema import HumanMessage, SystemMessage, Document, AIMessage
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.tools import DuckDuckGoSearchRun
from langgraph.types import Command
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, END, MessagesState, START
from typing import TypedDict, Optional, Literal, List, Dict
from typing_extensions import TypedDict

import datetime
import warnings
import os
import re

In [2]:
warnings.filterwarnings("ignore")

In [3]:
os.environ["GIGACHAT_CREDENTIALS"] = "Yjc1YWZhNTItMzYwYS00NmU4LTk4YjctZjU4YzAwMDIyMGJmOjJhNWE3YmVhLTAyYzctNGJhNy05NWE3LWEzY2YwNGQzYzZiNw=="

In [4]:
with open("../sources/sa_instruction.md", "r") as f:
    sa_instruction = f.read()

def save_result(result):
    format_ = "%Y_%m_%dT%H_%M_%S"
    datetime_mark = datetime.datetime.now().strftime(format_)
    
    with open(f"../tmp/sa_{datetime_mark}.md", "w") as f:
        f.write(result["sa_requirements"])

In [197]:
llm = GigaChat(
    model="GigaChat-2-Max",
    verify_ssl_certs=False,
    profanity_check=False,
    streaming=False,
    max_tokens=8192,
    temperature=0.3,
    repetition_penalty=1.01,
    timeout=60
)

In [6]:
tools = []

In [230]:
analitic_agent = create_react_agent(
    llm,
    tools=tools
)

generator_agent = create_react_agent(
    llm,
    tools=tools
)

In [231]:
class AgentState(TypedDict):
    messages: List
    task: str
    elements: str
    ba_requirements: str
    sa_requirements: str
    recommendations: str
    result: str

In [232]:
def analitic_node(state:AgentState)->Command[Literal["generator_agent"]]:
    """Агент-аналитик составляет список элементов системы."""
    print("Status: analitic_node")
    request = analitic_prompt.format(
        task=state["task"], 
        ba_requirements=state["ba_requirements"]
    )
    request = {
        "messages": [HumanMessage(content=request)]
    }
    response = analitic_agent.invoke(request)
    if isinstance(response, dict):
        result = response["messages"][-1].content
    return Command(
        update={
            "elements": response
        },
        goto="generator_agent"
    )

def generator_node(state:AgentState)->Command[Literal["validator_agent"]]:
    """Агент-генератор составляет системные требования, основываясь на бизнес-требованиях и элементах системы."""
    print("Status: generator_node")
    ba_requirements=state["ba_requirements"]
    elements=state["elements"]
    if "messages" in state and state["messages"]:
        old_messages = state["messages"]
        request = state["messages"][-1].content
    else:
        old_messages = []
        request = generator_prompt.format(
            sa_instruction=sa_instruction,
            ba_requirements=ba_requirements, 
            elements=elements, 
        )
    request = {
        "messages": [HumanMessage(content=request)]
    }
    response = generator_agent.invoke(request, config=config)
    if isinstance(response, dict):
        result = response["messages"][-1].content
    print(result)
    goto = "validator_agent"
    messages = old_messages + [HumanMessage(content=result, name="Аналитик")]
    return Command(
        update={
            "sa_requirements": result,
            "messages": messages
        },
        goto=goto
    )

def validator_node(state:AgentState)->Command[Literal["generator_agent", END]]:
    """Агент-генератор составляет системные требования, основываясь на бизнес-требованиях и элементах системы."""
    print("Status: validator_node")
    sa_requirements=state["sa_requirements"]
    ba_requirements=state["ba_requirements"]
    prompt = validator_prompt.format(
        ba_requirements=ba_requirements,
        sa_instruction=sa_instruction,
        format=format_
    )
    system = SystemMessage(content=prompt)
    request = [system] + state["messages"]
    
    validator_msgs_fount = len(list(filter(lambda x: x.name=="Валидатор", state["messages"])))
    response = llm.invoke(request)
    
    response = response.content

    print(f"RESPONSE: {response}")
    match_json = re.search(r'```json\s*(.*?)\s*```', response, re.DOTALL)
    if match_json:
        response = eval(match_json.group(1))
    else:
        response = {"status":"SUCCESS", "instructions":""}
    
    result = response["instructions"]
    print(f"STATUS: {response['status']}\n\nVALIDATOR: {result}")
    
    validator_msgs_fount = len(list(filter(lambda x: x.name=="Валидатор", state["messages"])))
    
    if response["status"]=="SUCCESS" or validator_msgs_fount+1 > 3:
        goto = END
    else:
        goto = "generator_agent"
    
    return Command(
        update={
            "messages": state["messages"] + [HumanMessage(content=result, name="Валидатор")]
        },
        goto=goto
    )

In [233]:
import uuid

In [234]:
config = {"configurable": {"thread_id": "thread_id789"}}

In [235]:
builder = StateGraph(state_schema=AgentState)

builder.set_entry_point("analitic_agent")
builder.add_node("analitic_agent", analitic_node)
builder.add_node("generator_agent", generator_node)
builder.add_node("validator_agent", validator_node)

graph = builder.compile()

In [236]:
analitic_prompt = """Ты - превосходный системный аналитик. Тебе дается задача и написанные для этой задачи бизнес-требования. 
Твоя задача - выделить основные элементы системы. Каждый элемент системы состоит из следующих компонентов:
- актор (пользовательская роль);
- действие (выполняемое актором);
- объект (над чем производится действие);
- результат (что должно произойти после выполнения действия).
Дай ответ в виде списка выделенных элементов.


Задача: {task}
Бизнес-требования: {ba_requirements}
"""

In [237]:
generator_prompt = """Ты - превосходный системный аналитик. Тебе даются бизнес-требования {ba_requirements} и список основных элементов системы {elements}. 
Каждый элемент системы состоит из следующих компонентов:
- актор (пользовательская роль);
- действие (выполняемое актором);
- объект (над чем производится действие);
- результат (что должно произойти после выполнения действия).
Твоя задача - написать системные требования. Для этого воспользуйся следующей инструкцией:\n
{sa_instruction}.
"""

In [238]:
validator_prompt = """Ты - Валидатор. Ты действуешь в паре с аналитиком. Ты умеешь очень хорошо критиковать системные требования для достижения лучшего результата. 
Тебе даются бизнес-требования {ba_requirements} и сгенерированные на их основе системные требования. 
Также у тебя есть инструкция по написанию системных требований: 
{sa_instruction}.
Проверь системные требования на логику и соответствие инструкции.
Если можно улучшить системные требования, напиши свои замечания и верни их на дорабоку, ответив FAIL. 
Если Аналитик хорошо выполнил свою работу и замечаний нет, ответь SUCCESS.

Ответ дай в следующем формате json:
```json
{format}
```
"""

In [239]:
format_ ="""{
    "instructions": "Здесь укажи свои рекомендации или оставь строку пустой, если рекомендаций нет",
    "status": "FAIL" если требуется доработка иначе "SUCCESS"
}"""

In [240]:
builder = StateGraph(state_schema=AgentState)

builder.set_entry_point("analitic_agent")
builder.add_node("analitic_agent", analitic_node)
builder.add_node("generator_agent", generator_node)
builder.add_node("validator_agent", validator_node)

graph = builder.compile()

In [241]:
with open("../tmp/ba_2025_05_18T04_33_30.md", "r") as f:
    ba_requirements = f.read()

task = """Требуется создать систему автозаполнения заявок инженеров сопровождения. Как должна работать система:
Есть база данных "ПКАП", с которой можно взаимодействовать через сторонний сервис OpenShift. Сервис Openshift извлекает данные из БД и отдает их системе.
Данные представляют собой заявку по инциденту, которые необходимо заполнить. Система должна проанализировать заявку, выделить причины инциденда и способ его закрытия.
Система должна сравнить проанализированные данные с аналогичными примерами, которые у нее есть. В случае, если найден похожий пример, заполнить заявку основываясь на примере и результаете своего анализа."""

In [242]:
initial_state = {
    "task": task,
    "ba_requirements": ba_requirements
}

In [244]:
# generator_agent.checkpointer.get(config)

In [245]:
result = graph.invoke(initial_state)

Status: analitic_node
Status: generator_node
# Системные требования к задаче: Автоматизация обработки заявок в OpenShift

## 1. Знакомство с бизнес-контекстом и бизнес-требованиями, их уточнение

Проект направлен на автоматизацию обработки заявок в системе OpenShift с целью повышения эффективности работы сотрудников и сокращения времени на обработку инцидентов. В рамках проекта будет разработана система, состоящая из пяти агентов, каждый из которых выполняет свою задачу: анализ заявки, поиск аналогичных примеров, заполнение заявки, проверка корректности и координация работы всех агентов. Система будет взаимодействовать с OpenShift и базой данных "ПКАП" через RAG и API.

## 2. Определение ролей пользователей и приложений. Верхнеуровневое проектирование архитектуры

### Определение ролей пользователей и приложений

- **Сотрудник службы поддержки**: отправляет заявку в OpenShift.
- **Агент-анализатор**: анализирует заявку.
- **Агент-сравниватель**: ищет аналогичные примеры в базе данных "

In [246]:
save_result(result)